Node representation learning with Metapath2Vec

An example of implementing the Metapath2Vec representation learning algorithm using components from the stellargraph and gensim libraries.

References

1.Metapath2Vec: Scalable Representation Learning for Heterogeneous Networks. Yuxiao Dong, Nitesh V. Chawla, and Ananthram Swami. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 135–144, 2017. ([link](https://ericdongyx.github.io/papers/KDD17-dong-chawla-swami-metapath2vec.pdf))

2.Distributed representations of words and phrases and their compositionality. T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. In Advances in Neural Information Processing Systems (NIPS), pp. 3111-3119, 2013. ([link](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf))

3.Gensim: Topic modelling for humans. ([link](https://radimrehurek.com/gensim/))

In [1]:
!pip install -q stellargraph[demos]==1.2.1

In [2]:
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [3]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
import itertools
from IPython.display import display, HTML

%matplotlib inline

Load the dataset (See the "[Loading from Pandas](https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/basics/loading-pandas.ipynb#scrollTo=SNSSSEjhwzke)" demo for details on how data can be loaded.)

In [4]:
author_paper_all_with_year = pd.read_csv("../input/ee226-project-dataset/author_paper_all_with_year.csv")
author_paper = author_paper_all_with_year[["author_id","paper_id"]]
paper_reference = pd.read_csv("../input/ee226-project-dataset/paper_reference.csv")

In [5]:
def load(author_paper, paper_reference): 
    authorship = author_paper.rename(columns={'author_id': 'source', 'paper_id': 'target'}) # change 'author_paper' to 'authorship' to keep 'author_paper' unchanged for further use
    paper_reference = paper_reference.rename(columns={'paper_id': 'source', 'reference_id': 'target'})
    

    # build edges   
    # add prefix
    def a(x):
        return "author-" + x.astype(str)
        
    def p(x):
        return "paper-" + x.astype(str)
            
    authorship["source"] = a(authorship["source"])
    authorship["target"] = p(authorship["target"])
    paper_reference = p(paper_reference)
    
    # add author-author edges
    a_id = author_paper['author_id'].tolist()
    p_id = author_paper['paper_id'].tolist()
    author_author1 = []
    author_author2 = []
    mat = []

    for j in range(len(list(set(p_id)))):
        tmp = []
        for i in range(len(p_id)):
            if p_id[i] == j:
                tmp.append(a_id[i])
        cc = list(itertools.combinations(tmp, 2))
        for k in range(len(cc)):
            author_author1.append('author-'+str(cc[k][0]))
            author_author2.append('author-'+str(cc[k][1]))
    
    author_author = {"source":author_author1,"target":author_author2}
    author_author = pd.DataFrame(author_author)
    
    edges = authorship.append(paper_reference, ignore_index=True)
    edges = edges.append(author_author, ignore_index=True)
#     edges = {"author-paper": authorship, "paper_reference": paper_reference, "author-author": author_author}
#     (the above line makes an error about ids appearing more than once)
    
    
    # build nodes
    def nodes(x_id):
        index = pd.DataFrame(author_paper[x_id].values)
        N = index.max()
        x_i = range(N[0]+1)
        x_i = pd.DataFrame(index=x_i)
        return x_i
    
    author_ids = nodes("author_id")
    authors = author_ids.set_index(
        "author-" + author_ids.index.astype(str)
    )
    
    paper_ids = nodes("paper_id")
    papers = paper_ids.set_index(
        "paper-" + paper_ids.index.astype(str)
    )
    
    nodes = {"author": authors, "paper": papers}
    
    
    return sg.StellarGraph(nodes, edges)

In [6]:
g = load(author_paper, paper_reference)
print(g.info())

StellarGraph: Undirected multigraph
 Nodes: 66865, Edges: 299093

 Node types:
  author: [42614]
    Features: none
    Edge types: author-default->author, author-default->paper
  paper: [24251]
    Features: none
    Edge types: paper-default->author, paper-default->paper

 Edge types:
    author-default->author: [181338]
        Weights: all 1 (default)
        Features: none
    author-default->paper: [93230]
        Weights: all 1 (default)
        Features: none
    paper-default->paper: [24525]
        Weights: all 1 (default)
        Features: none


The Metapath2Vec algorithm

The Metapath2Vec algorithm introduced in [1] is a 2-step representation learning algorithm. The two steps are:

Use uniform random walks to generate sentences from a graph. A sentence is a list of node IDs. The set of all sentences makes a corpus. The random walk is driven by a metapath that defines the node type order by which the random walker explores the graph.

The corpus is then used to learn an embedding vector for each node in the graph. Each node ID is considered a unique word/token in a dictionary that has size equal to the number of nodes in the graph. The Word2Vec algorithm [2] is used for calculating the embedding vectors.

Corpus generation using random walks

The stellargraph library provides an implementation for uniform, first order, random walks as required by Metapath2Vec. The random walks have fixed maximum length and are controlled by the list of metapath schemas specified in parameter metapaths.

A metapath schema defines the type of node that the random walker is allowed to transition to from its current location. In the stellargraph implementation of metapath-driven random walks, the metapath schemas are given as a list of node types under the assumption that the input graph is not a multi-graph, i.e., two nodes are only connected by one edge type.

See [1] for a detailed description of metapath schemas and metapath-driven random walks.

For the dataset we use the following 4 metapaths.

"author", "paper", "author"

"author", "paper", "paper", "author"

"author","author"

"author","paper","paper","author","author"

In [7]:
walk_length = 100  # maximum length of a random walk to use throughout this notebook

# specify the metapath schemas as a list of lists of node types.
metapaths = [
    ["author", "paper", "author"],
    ["author", "paper", "paper", "author"],
    ["author","author"],
    ["author","paper","paper","author","author"]
]

In [8]:
from stellargraph.data import UniformRandomMetaPathWalk

# Create the random walker
rw = UniformRandomMetaPathWalk(g)

walks = rw.run(
    nodes=list(g.nodes()),  # root nodes
    length=walk_length,  # maximum length of a random walk
    n=1,  # number of random walks per root node
    metapaths=metapaths,  # the metapaths
)

print("Number of random walks: {}".format(len(walks)))

Number of random walks: 170456


Representation Learning using Word2Vec

We use the Word2Vec [2] implementation in the free Python library gensim [3] to learn representations for each node in the graph.

We set the dimensionality of the learned embedding vectors to 128 as in [1].

In [9]:
from gensim.models import Word2Vec

model = Word2Vec(walks, vector_size=128, window=5, min_count=0, sg=1, workers=12)

In [10]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index_to_key  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [g.node_type(node_id) for node_id in node_ids]

In [11]:
print(len(node_ids))

66865


Visualise Node Embeddings

We retrieve the Word2Vec node embeddings that are 128-dimensional vectors and then we project them down to 2 dimensions using the t-SNE algorithm.

In [12]:
# transform = TSNE  # PCA

# trans = transform(n_components=2)
# node_embeddings_2d = trans.fit_transform(node_embeddings)

In [13]:
# # draw the points
# label_map = {l: i for i, l in enumerate(np.unique(node_targets))}
# node_colours = [label_map[target] for target in node_targets]

# plt.figure(figsize=(20, 16))
# plt.axes().set(aspect="equal")
# plt.scatter(node_embeddings_2d[:, 0], node_embeddings_2d[:, 1], c=node_colours, alpha=0.3)
# plt.title("{} visualization of node embeddings".format(transform.__name__))
# plt.show()

Downstream task

The node embeddings calculated using Metapath2Vec can be used as feature vectors in a downstream task such as node classification and link prediction.

In [14]:
# extract author ids and corresponding embeddings
indices = []
for i in range(len(node_targets)):
    if node_targets[i] == 'author':
        indices.append(i)
        
author_ids = list( node_ids[i] for i in indices )
author_embeddings = node_embeddings[[indices[0]]]
for i in indices[1:]:
    author_embeddings = np.append(author_embeddings, node_embeddings[[i]], axis=0)

In [15]:
# match author embeddings and labels
author_label = pd.read_csv("../input/ee226-project-dataset/labeled_papers_with_authors.csv")
author_id = pd.DataFrame(index=author_label["author_id"].values)
author_id_prefix = author_id.set_index(
    "author-" + author_id.index.astype(str)
)
author_id_prefix_list = author_id_prefix.index.tolist()
label = author_label["label"].values


num_author_label = len(np.unique(author_label["author_id"].values))
num_label = len(np.unique(author_label["label"].values))
label_list = np.zeros((num_author_label, num_label))


author_id_embeddings = author_embeddings[[0]]
author_embedded = []

for i in range(len(author_id_prefix_list)):    
    num = int(author_id_prefix_list[i].split('-')[1])
    for j in range(len(author_ids)):
        if author_ids[j] == author_id_prefix_list[i]:
            if num not in author_embedded:
                author_embedded.append(num)
                author_id_embeddings = np.append(author_id_embeddings, author_embeddings[[j]], axis=0)
            label_list[num][label[i]] = 1
            
author_id_embeddings = np.delete(author_id_embeddings, 0,0)

In [16]:
X = author_id_embeddings
y = label_list

print(X.shape,y.shape)

(13449, 128) (13449, 10)


In [17]:
# match authors to predict and their embeddings
author_to_pred = pd.read_csv("../input/ee226-project-dataset/authors_to_pred.csv")
author_to_pred_id = pd.DataFrame(index=author_to_pred["author_id"].values)
author_to_pred_prefix = author_to_pred_id.set_index(
    "author-" + author_to_pred_id.index.astype(str)
)
author_to_pred_prefix_list = author_to_pred_prefix.index.tolist()
author_to_pred_id_list = author_to_pred_id.index.tolist()

author_to_pred_ed = author_embeddings[[0]]

for i in range(len(author_to_pred_prefix_list)):
    for j in range(len(author_ids)):
        if author_to_pred_prefix_list[i] == author_ids[j]:
                author_to_pred_ed = np.append(author_to_pred_ed, author_embeddings[[j]], axis=0)
                
author_to_pred_ed = np.delete(author_to_pred_ed, 0, 0)
X_author_to_pred = author_to_pred_ed

In [18]:
X_author_to_pred.shape

(37066, 128)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
normalizer = preprocessing.Normalizer().fit(X)
X_normalized = normalizer.transform(X)

scaler = preprocessing.StandardScaler().fit(X_author_to_pred)
X_to_pred_scaled = scaler.transform(X_author_to_pred)
normalizer = preprocessing.Normalizer().fit(X_author_to_pred)
X_to_pred_normalized = normalizer.transform(X_author_to_pred)

In [20]:
# # use DecisionTreeClassifier for prediction on multi-label classification

# X_train, X_test, Y_train ,Y_test = train_test_split(X_normalized, y, test_size=0.8)

# cls = DecisionTreeClassifier()
# cls.fit(X_train, Y_train)

# from sklearn import metrics

# Y_pred = cls.predict(X_test)

# print(metrics.f1_score(Y_test, Y_pred, average="macro"))
# print(metrics.f1_score(Y_test, Y_pred, average="micro"))

In [21]:
# cls.fit(X_normalized, y)
# label_pred = cls.predict(X_to_pred_normalized)

In [22]:
# use mlp for prediction on multi-label classification
from keras.models import Sequential
from keras.layers import Dense, Dropout

# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(200, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [23]:
# X_train, X_test, Y_train ,Y_test = train_test_split(X_normalized, y, test_size=0.8)

# n_inputs, n_outputs = X.shape[1], y.shape[1]
# # get model
# model = get_model(n_inputs, n_outputs)
# # fit the model on all data
# model.fit(X_train, Y_train, verbose=0, epochs=200)
# yhat0 = model.predict(X_test)

# # round probabilities to class labels
# # yhat0_1 = yhat1.round()
# yhat0_1 = yhat0>0.1
# yhat0_1 = yhat0_1.astype(int)
# # choose a label for all-zero rows
# yhat0_2 = yhat0_1==0
# for i in range(yhat0_1.shape[0]):
#     if bool(yhat0_2[i].all())==True:
#         yhat0_1[i, np.argmax(yhat1[i])] = 1

# f1_macro = metrics.f1_score(Y_test, yhat0_1, average="macro")
# f1_micro = metrics.f1_score(Y_test, yhat0_1, average="micro")
# print('>%.3f %.3f' % (f1_macro, f1_micro))

In [24]:
n_inputs, n_outputs = X.shape[1], y.shape[1]

# get model
model = get_model(n_inputs, n_outputs)

# fit the model on all data
model.fit(X_normalized, y, verbose=0, epochs=150)

# make a prediction for new data
yhat = model.predict(X_to_pred_normalized)

In [25]:
# round probabilities to class labels
yhat1 = yhat>0.09
yhat1 = yhat1.astype(int)

# choose a label for all-zero rows
yhat2 = yhat1==0
for i in range(yhat1.shape[0]):
    if bool(yhat2[i].all())==True:
        yhat1[i, np.argmax(yhat[i])] = 1

# replace author embeddings with ground truths when author id is smaller than 13449
y = y.astype(int)
for i in range(len(author_to_pred_id_list)):
    if author_to_pred_id_list[i] <= 13448:
        yhat1[i,:] = y[author_to_pred_id_list[i],:]

In [26]:
label_pred_dataframe = pd.DataFrame(columns=['labels'])
for i in range(yhat1.shape[0]):
    label_idx = np.nonzero(yhat1[i, :])
    label_str = ' '.join(str(j) for j in label_idx[0])
    label_pred_dataframe.loc[i] = label_str

In [27]:
res = pd.concat([author_to_pred,label_pred_dataframe],axis=1,ignore_index=False)

In [28]:
res.to_csv('./pred.csv', index=None)